In [4]:
pip install pandas numpy matplotlib seaborn scikit-learn openpyxl biopython

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

genoma='C:/Users/judit/OneDrive/Escritorio/MÁSTER/TFM/Bases de datos/BVBRC_genome_amr.xlsx'
df_genoma = pd.read_excel(genoma)

genes='C:/Users/judit/OneDrive/Escritorio/MÁSTER/TFM/Bases de datos/BVBRC_sp_gene.xlsx'
df_genes = pd.read_excel(genes)

In [ ]:
from Bio import SeqIO
genes_fasta='C:/Users/judit/OneDrive/Escritorio/MÁSTER/TFM/Bases de datos/BVBRC_genome_feature.fasta'

for record in SeqIO.parse(genes_fasta, "fasta"):
    print("ID:", record.id)
    print("Descripción:", record.description)
    print("Secuencia de la proteína:", record.seq)
    print("Longitud de la secuencia:", len(record.seq))
    print("-" * 50)

In [10]:
# Seleccionar las columnas clave
df_genoma = df_genoma[['Genome Name', 'Resistant Phenotype']]
df_genes = df_genes[['Genome Name', 'BRC ID']]

In [11]:
#Convertir resistencia a binario (1=Resistente, 0=Susceptible)
df_genoma['Resistance'] = df_genoma['Resistant Phenotype'].apply(lambda x: 1 if 'Carbapenem' in str(x) else 0)
df_genoma = df_genoma[['Genome Name', 'Resistance']]

In [33]:
# Crear matriz de presencia/ausencia de genes
gene_presence = df_genes.pivot_table(index='Genome Name', columns='BRC ID', aggfunc=lambda x: 1, fill_value=0)

# Contar en cuántos genomas aparece cada gen
gene_counts = gene_presence.sum(axis=0)

df_genoma["Genome Name"] = df_genoma["Genome Name"].str.strip().str.lower()
gene_presence.index = gene_presence.index.str.strip().str.lower()

# Filtrar genes que están en al menos el 1% de los genomas
threshold = len(df_genoma) * 0.01
selected_genes = gene_counts[gene_counts > threshold].index
gene_presence_filtered = gene_presence[selected_genes]

df_final = df_genoma.merge(gene_presence, left_on='Genome Name', right_index=True, how='left').fillna(0)

In [36]:
from sklearn.model_selection import train_test_split

# Separar en conjuntos de entrenamiento y prueba
X = df_final.drop(columns=['Genome Name', 'Resistance'])
y = df_final['Resistance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [37]:
# Verificar dimensiones
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (74456, 5341)
Tamaño del conjunto de prueba: (31910, 5341)


In [38]:
print("Tamaño del conjunto final:", df_final.shape)

Tamaño del conjunto final: (106366, 5343)
